# Dies ist der Klausurtest
Aufgabe: erstelle ein Programm, das die bereitgestellten CSV-Datei einliest
und lowercase am Bildschirm ausgibt, und Zeile f. Zeile mit der jeweiligen Anzahl von Worten
in der jeweiligen Zeile am Anfang.
Bsp: Die Zeile "hEllo World" soll folgendermassen am Bildschirm ausgegeben werden:

2 hello world

In [3]:
from pyspark.sql import SparkSession

# Create SparkSession
spark = SparkSession.builder\
    .master("local[1]")\
    .appName("Datenbanken mit Spark")\
    .getOrCreate() # getOrCreate liefert existierende Session, wenn es schon eine gibt

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/19 10:12:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark

In [8]:
text = spark.read.text("/home/student/Desktop/Klausurtest.csv")

In [7]:
text

DataFrame[value: string]

In [15]:
text.show(100, truncate=False)

+------------------------------+
|value                         |
+------------------------------+
|5 mal werden Wir noch wach,   |
|Heißa dann ist Weihnachtsnacht|
+------------------------------+



In [37]:
# DESIGNENTSCHEIDUNG
# Nur Wörter ausgegeben werden
# Zahlen werden ausgeschlossen
# Die Buchstaben können Groß- oder Kleingeschrieben sein
import pyspark.sql.functions as F
lines = text.select(split(text.value, "[^a-z,A-Z,ß]").alias("Zeile"))
lines.show(100,truncate=False)

+-------------------------------------+
|Zeile                                |
+-------------------------------------+
|[, , mal, werden, Wir, noch, wach,, ]|
|[Heißa, dann, ist, Weihnachtsnacht]  |
+-------------------------------------+



In [63]:
# Leerzeichen werden noch mitgezählt!
lines = lines.withColumn("Anzahl", F.size(F.col("Zeile")))
lines.show(100, truncate=False)

+------+-------------------------------------+
|Anzahl|Zeile                                |
+------+-------------------------------------+
|8     |[, , mal, werden, Wir, noch, wach,, ]|
|4     |[Heißa, dann, ist, Weihnachtsnacht]  |
+------+-------------------------------------+



In [58]:
# Spalten vertauschen
lines = lines.select(lines.Anzahl, lines.Zeile)
lines.show(100, truncate=False)

+------+-------------------------------------+
|Anzahl|Zeile                                |
+------+-------------------------------------+
|8     |[, , mal, werden, Wir, noch, wach,, ]|
|4     |[Heißa, dann, ist, Weihnachtsnacht]  |
+------+-------------------------------------+



In [74]:
# DESIGNENTSCHEIDUNG
# Leerzeichen noch entfernen
non_empty = lines.select(F.array_remove(F.col("Zeile"), "")).withColumnRenamed("array_remove(Zeile, )", "Zeile")
counted = non_empty.withColumn("Anzahl", F.size(F.col("Zeile")))
counted.select(counted.Anzahl, counted.Zeile).show(truncate=False)

+------+-----------------------------------+
|Anzahl|Zeile                              |
+------+-----------------------------------+
|5     |[mal, werden, Wir, noch, wach,]    |
|4     |[Heißa, dann, ist, Weihnachtsnacht]|
+------+-----------------------------------+

